In [1]:
import nltk
import string
import numpy as np
import pandas as pd
import os
from nltk.corpus import stopwords


from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.utils import to_categorical

/home/user/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
df = pd.read_csv('/home/user/Desktop/timesofindiatrainfiles/timesofindia_train.csv', sep='\t')


In [3]:
embeddings_index = dict()
f = open('/home/user/Desktop/timesofindiatrainfiles/glove.6B/glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [4]:

df_sent = (df['sentence'].str.len() > 10)
ne_df = df.loc[df_sent]
df = ne_df
df = df[~df.sentence.str.startswith('https')]
df = df[~df.sentence.str.startswith('url : ')]

In [6]:
df.sentence.head(20)

0     APCC president warns member over inside fighti...
1     "anyone in my position would have done the sam...
2     speaking to reporters after receiving a memora...
3                         they should have informed me.
4     they also did not inform me that the dcc had i...
5     the apcc president made it clear that he would...
6     on the demand from party mla shikhamani to sus...
7     Doctors to stop OPD service TNN | Aug 5, 2001,...
8     the karnataka state junior doctors association...
9     a relay hunger strike will begin from tuesday ...
10    the junior doctors of mysore city will take ou...
11     a rasta roko will be held in bijapur on tuesday.
12    student organisations like abvp, sufi, suci an...
13    they will join us in our struggle, the associa...
14    withdrawal of opd by the striking doctors from...
19    ramaiah and other major hospitals all over the...
20    it is learnt that the rajiv gandhi university ...
22    Teachers blink first, call off strike TNN 

In [7]:
import nltk

w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

In [8]:
def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]

df.sentence = df.sentence.apply(lemmatize_text)

In [9]:
df.sentence.head(20)

0     [APCC, president, warns, member, over, inside,...
1     ["anyone, in, my, position, would, have, done,...
2     [speaking, to, reporter, after, receiving, a, ...
3                   [they, should, have, informed, me.]
4     [they, also, did, not, inform, me, that, the, ...
5     [the, apcc, president, made, it, clear, that, ...
6     [on, the, demand, from, party, mla, shikhamani...
7     [Doctors, to, stop, OPD, service, TNN, |, Aug,...
8     [the, karnataka, state, junior, doctor, associ...
9     [a, relay, hunger, strike, will, begin, from, ...
10    [the, junior, doctor, of, mysore, city, will, ...
11    [a, rasta, roko, will, be, held, in, bijapur, ...
12    [student, organisation, like, abvp,, sufi,, su...
13    [they, will, join, u, in, our, struggle,, the,...
14    [withdrawal, of, opd, by, the, striking, docto...
19    [ramaiah, and, other, major, hospital, all, ov...
20    [it, is, learnt, that, the, rajiv, gandhi, uni...
22    [Teachers, blink, first,, call, off, strik

In [10]:
sent_list = []
label_list = []

sent_list = df["sentence"].tolist()
label_list = df["etype"].tolist()

print(len(label_list))
print(label_list[0])

1323
0


In [11]:
### Create sequence
first_1000 = df['sentence'][0:1000]
label_list = df['etype'][0:1000]
vocabulary_size = 20000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(first_1000)

sequences = tokenizer.texts_to_sequences(first_1000)
data = pad_sequences(sequences, maxlen=200)

In [12]:
embedding_matrix = np.zeros((vocabulary_size, 100))
for word, index in tokenizer.word_index.items():
    if index > vocabulary_size - 1:
        break
    else:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector

In [13]:
model_glove = Sequential()
model_glove.add(Embedding(vocabulary_size, 100, input_length=200, weights=[embedding_matrix], trainable=False))
model_glove.add(LSTM(100))
model_glove.add(Dense(1, activation='sigmoid'))
model_glove.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [14]:
model_glove.fit(data, np.array(label_list), validation_split=0.0, epochs=4)

Epoch 1/4
1000/1000 [==============================] - 7s 7ms/step - loss: 0.5817 - acc: 0.7290
Epoch 2/4
1000/1000 [==============================] - 5s 5ms/step - loss: 0.5404 - acc: 0.7430
Epoch 3/4
1000/1000 [==============================] - 5s 5ms/step - loss: 0.4831 - acc: 0.7800
Epoch 4/4
1000/1000 [==============================] - 5s 5ms/step - loss: 0.4165 - acc: 0.8110


In [15]:
model_cnn = Sequential()
model_cnn.add(Embedding(vocabulary_size, 100, input_length=200, weights=[embedding_matrix], trainable=False))
model_cnn.add(Conv1D (kernel_size = (200), filters = 20,  activation='relu'))
model_cnn.add(Flatten())
model_cnn.add(Dense(1, activation='sigmoid'))
model_cnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [35]:
model_cnn.fit(data, np.array(label_list), validation_split=0.0, epochs=4)

Epoch 1/4
1000/1000 [==============================] - 1s 1ms/step - loss: 0.5702 - acc: 0.7350
Epoch 2/4
1000/1000 [==============================] - 0s 354us/step - loss: 0.3937 - acc: 0.8340
Epoch 3/4
1000/1000 [==============================] - 0s 369us/step - loss: 0.2977 - acc: 0.8950
Epoch 4/4
1000/1000 [==============================] - 0s 355us/step - loss: 0.2209 - acc: 0.9260


In [78]:
from keras.layers import GRU

model_rnn = Sequential()
model_rnn.add(Embedding(vocabulary_size, 100, input_length=200, weights=[embedding_matrix], trainable=False))
model_rnn.add(GRU(64, activation='relu'))
model_rnn.add(Dense(1, activation='sigmoid'))
model_rnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])



In [79]:
model_rnn.fit(data, np.array(label_list), validation_split=0.0, epochs=4)

Epoch 1/4
1000/1000 [==============================] - 5s 5ms/step - loss: 0.5932 - acc: 0.7190
Epoch 2/4
1000/1000 [==============================] - 3s 3ms/step - loss: 0.5571 - acc: 0.7320
Epoch 3/4
1000/1000 [==============================] - 3s 3ms/step - loss: 0.5330 - acc: 0.7340
Epoch 4/4
1000/1000 [==============================] - 3s 3ms/step - loss: 0.5025 - acc: 0.7630


In [80]:
### Create sequence
first_1000 = df['sentence'][1000:]
label_list = df['etype'][1000:]
vocabulary_size = 2333
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(first_1000)

sequences = tokenizer.texts_to_sequences(first_1000)
data = pad_sequences(sequences, maxlen=200)

In [81]:
prediction_lstm = model_glove.predict(data)
prediction_lstm = [1 if pred>=0.5 else 0 for pred in prediction_lstm]

prediction_cnn = model_cnn.predict(data)
prediction_cnn = [1 if pred>=0.5 else 0 for pred in prediction_cnn]

prediction_rnn = model_rnn.predict(data)
prediction_rnn = [1 if pred>=0.5 else 0 for pred in prediction_rnn]

In [82]:
prediction_rnn[0:300]

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [83]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
print("LSTM F1 : ", f1_score(label_list, prediction_lstm))
print("CNN F1 : ", f1_score(label_list, prediction_cnn, average="weighted"))
print("RNN F1 : ", f1_score(label_list, prediction_rnn))



LSTM F1 :  0.196969696969697
CNN F1 :  0.6559509989161839
RNN F1 :  0.11666666666666667


In [19]:
accuracy_score(label_list, prediction_cnn)


0.6842105263157895

In [20]:
data[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])